# Submission

Arma `submission.csv` para subir a Kaggle.

In [1]:
import sys
sys.path.append('..')

In [2]:
from tqdm import tqdm

from lib.data import load_all_invoices, load_extracted_samples
from lib.details import find_detail_lines
from lib.invoice import find_invoice
from lib.line_classification import is_professional

In [3]:
all_invoices = load_all_invoices()
len(all_invoices)

8411

In [4]:
samples = load_extracted_samples()
len(samples)

8411

In [5]:
invoice_dict = {}
for invoice in all_invoices:
    invoice_dict[invoice] = None

for sample in tqdm(samples):
    filename = sample["filename"][:-4] # .pdf
    
    invoice = find_invoice(sample)
    detail_lines = find_detail_lines(sample)
    
    total = 0
    for line in detail_lines:
        if line is not None:
            if is_professional(line["desc"]):
                total += line["total"]

    if total == 0:
        total = None

    if invoice is not None and total is not None:
        invoice_dict[invoice] = total

  0%|          | 20/8411 [00:04<32:42,  4.28it/s] 

In [ ]:
all_totals = [total for total in invoice_dict.values() if total is not None]
avg = sum(all_totals) / len(all_totals)
avg = round(avg, 2)
len(all_totals), len(all_totals) / len(samples), avg

In [ ]:
# write csv
with open("submission.csv", 'w') as f:
    f.write('Invoice Number,Total Charged\n')
    for invoice, total in invoice_dict.items():

        if total == None:
            total = 9783.46 # avg

        f.write(f'{invoice},{total}\n')